In [1]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
import os
os.getcwd()

'C:\\Users\\PC\\OneDrive\\Data Science\\Weak-supervision'

In [3]:
# 연도별로 엑셀자료에 정렬해놓은 데이터 가져오기 (2010 ~ 2019년)
# 회사코드 Symbol에서 앞에 'A'제거하고 숫자만 가져오는 과정 ; convert symbol to eliminate useless 'A' (ex. A000010 -> 000010)
def symbol_convertor(str_):
    return str_[1:]

data_2010 = pd.read_excel("코스피200 편입여부.xlsx", sheet_name="2010-01-31")
data_2010

,Symbol,company_name,2010-01-31 00:00:00
0,A005930,삼성전자,Y
1,A000660,SK하이닉스,Y
2,A035420,NAVER,Y
3,A005380,현대차,Y
4,A051910,LG화학,Y
...,...,...,...
196,A053000,우리금융,Y
197,A064420,케이피케미칼,Y
198,A067250,STX조선해양,Y
199,A068870,LG생명과학,Y


In [10]:
# 전체 기업코드 변환
symbol_origin = list(data_2010.loc[:199,'Symbol'].values)
symbol_origin = list(map(symbol_convertor, symbol_origin))
data_2010.loc[:199, 'Symbol'] = symbol_origin
data_2010

,Symbol,company_name,2010-01-31 00:00:00
0,005930,삼성전자,Y
1,000660,SK하이닉스,Y
2,035420,NAVER,Y
3,005380,현대차,Y
4,051910,LG화학,Y
...,...,...,...
196,053000,우리금융,Y
197,064420,케이피케미칼,Y
198,067250,STX조선해양,Y
199,068870,LG생명과학,Y


In [11]:
# company_code & company_name

K_2010 = data_2010.iloc[:200,0:2]
K_2010_code = list(K_2010["Symbol"])
K_2010_name = list(K_2010["company_name"])
K_2010_new = K_2010.set_index("Symbol")     # 'company_code'를 index로 만들어 제외된 회사, 추가된 회사를 list로 만들기 위한 변수

In [12]:
# Dart의 IP차단을 막기 위해 갯수 설정 -> 반복문, 동적변수 할당으로 할 수 있음

K_2010_code_part = K_2010_code[:40]      # 40 ~ 80 , ... , 160 ~ 200
# K_2010_code_part

In [13]:
# 주소(url) setting

API_KEY = "9a9929b43fd4c79e06cc04d50eb34a70a0a576b1"
start_date = "20100101"
end_date = "20191231"
page_set = "100"   # 페이지당 건수
# auth : 발급받은 인증키(40자리)
# crp_cd : 공시대상회사의 종목코드(상장사:6자리 숫자) 또는 고유번호(기타법인:8자리 숫자)
# start_dt, end_dt : 검색시작,끝 접수일자
# dsn_tp : A001(사업보고서)

url_list = []
for crp in K_2010_code_part:
    company_code = crp   
    url = "http://dart.fss.or.kr/api/search.xml?auth=" + API_KEY + "&crp_cd=" + company_code + "&start_dt=" + start_date + "&end_dt=" + end_date + "&bsn_tp=A001&page_set=" + page_set  
    url_list.append(url)
# url_list

In [16]:
# 각 연도별로 코스피200종목 회사들의 2010 ~ 2019년 모든 사업보고서 리스트를 데이터프레임으로 반환

def data_extract():
    url_lists = url_list
    data_finals = pd.DataFrame()
    for url in url_lists:
        xmlsoup = BeautifulSoup(urlopen(url).read(), 'html.parser')
        te = xmlsoup.findAll("list")
        data = pd.DataFrame()
        for t in te:
            temp = pd.DataFrame(([[t.crp_cls.string, t.crp_nm.string, t.crp_cd.string, t.rpt_nm.string,
                              t.rcp_no.string, t.flr_nm.string, t.rcp_dt.string, t.rmk.string]]),
                              columns = ["crp_cls", "crp_nm", "crp_cd", "rpt_nm", "rcp_no", "flr_nm", "rcp_dt", "rmk"])
            data = pd.concat([data,temp])
            data_result = data
        # data_result = data_result[data_result['rmk'] != '정연']   # '정연'은 수정된 사업보고서가 제출되기 이전의 보고서이므로 제외
        data_finals = pd.concat([data_finals, data_result])
        data_finals = data_finals.reset_index(drop=True)
        # 중복된 사업보고서 삭제
        data_finals = data_finals.drop_duplicates(['crp_cls', 'crp_nm', 'crp_cd', 'rpt_nm', 'rcp_no', 'flr_nm', 'rcp_dt', 'rmk'], keep='first')
    return data_finals

In [ ]:
report_result = data_extract()
report_result

In [23]:
# json과 비교확인

company_code = ["053000","316140"]  # 신한, 한국금융지주
API_KEY = "9a9929b43fd4c79e06cc04d50eb34a70a0a576b1"
start_date = "20100101"
end_date = "20191231"
page_set = "100"   # 페이지당 건수
# auth : 발급받은 인증키(40자리)
# crp_cd : 공시대상회사의 종목코드(상장사:6자리 숫자) 또는 고유번호(기타법인:8자리 숫자)
# start_dt, end_dt : 검색시작,끝 접수일자
# dsn_tp : A001(사업보고서)
url_list = []
for crp in company_code:   
    url = "http://dart.fss.or.kr/api/search.xml?auth=" + API_KEY + "&crp_cd=" + crp + "&start_dt=" + start_date + "&end_dt=" + end_date + "&bsn_tp=A001&page_set=" + page_set  
    url_list.append(url)
url_list

def data_extract():
    url_lists = url_list
    data_finals = pd.DataFrame()
    for url in url_lists:
        xmlsoup = BeautifulSoup(urlopen(url).read(), 'html.parser')
        te = xmlsoup.findAll("list")
        data = pd.DataFrame()
        for t in te:
            temp = pd.DataFrame(([[t.crp_cls.string, t.crp_nm.string, t.crp_cd.string, t.rpt_nm.string,
                              t.rcp_no.string, t.flr_nm.string, t.rcp_dt.string, t.rmk.string]]),
                              columns = ["crp_cls", "crp_nm", "crp_cd", "rpt_nm", "rcp_no", "flr_nm", "rcp_dt", "rmk"])
            data = pd.concat([data,temp])
            data_result = data
        # data_result = data_result[data_result['rmk'] != '정연']   # '정연'은 수정된 사업보고서가 제출되기 이전의 보고서이므로 제외
        data_finals = pd.concat([data_finals, data_result])
        data_finals = data_finals.reset_index(drop=True)
        # 중복된 사업보고서 삭제
        data_finals = data_finals.drop_duplicates(['crp_cls', 'crp_nm', 'crp_cd', 'rpt_nm', 'rcp_no', 'flr_nm', 'rcp_dt', 'rmk'], keep='first')
    return data_finals

report_result_p = data_extract()
report_result_p

,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,E,우리금융지주,00375302,[기재정정]사업보고서 (2013.12),20140515001465,우리금융지주,20140515,연
1,E,우리금융지주,00375302,사업보고서 (2013.12),20140331002979,우리금융지주,20140331,정연
2,E,우리금융지주,00375302,사업보고서 (2012.12),20130401001236,우리금융지주,20130401,연
3,E,우리금융지주,00375302,[첨부정정]사업보고서 (2011.12),20120426000524,우리금융지주,20120426,연
4,E,우리금융지주,00375302,사업보고서 (2011.12),20120330001553,우리금융지주,20120330,정연
5,E,우리금융지주,00375302,사업보고서 (2010.12),20110331001704,우리금융지주,20110331,연
6,E,우리금융지주,00375302,사업보고서 (2009.12),20100330000023,우리금융지주,20100330,연


In [ ]:
url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20130329001399"

In [8]:
# csv file로 저장

report_result.to_csv(r'C:\Users\PC\Desktop/일단해봐.csv', encoding='euc-kr') # 한글깨짐 방지

In [2]:
# filtering한 데이터 가져오기

data = pd.read_excel(r"C:\Users\PC\Desktop\숭실대학교\19년도 3학년 2학기\핀랩\Weak-supervision\All report\2012/K_2012_5_2012.xlsx", sheet_name = 'Sheet1', encoding='euc-kr')
data = data.iloc[:-1,:]
data

,Unnamed: 0,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,6,Y,코오롱,2020.0,사업보고서 (2012.12),2.013040e+13,코오롱,20130401.0,연
1,17,Y,한국철강,104700.0,[기재정정]사업보고서 (2012.12),2.013051e+13,한국철강,20130510.0,NaN
2,28,Y,세아제강지주,3030.0,[기재정정]사업보고서 (2012.12),2.013051e+13,세아제강지주,20130513.0,연
3,38,Y,삼영전자공업,5680.0,사업보고서 (2012.12),2.013033e+13,삼영전자공업,20130326.0,연
4,48,Y,한솔홀딩스,4150.0,사업보고서 (2012.12),2.013033e+13,한솔홀딩스,20130329.0,연
5,58,Y,유수홀딩스,700.0,사업보고서 (2012.12),2.013040e+13,유수홀딩스,20130401.0,NaN
6,69,Y,STX,11810.0,사업보고서 (2012.12),2.013033e+13,STX,20130329.0,연
7,80,Y,STX엔진,77970.0,[기재정정]사업보고서 (2012.12),2.013051e+13,STX엔진,20130509.0,연
8,90,Y,디와이,13570.0,사업보고서 (2012.12),2.013033e+13,디와이,20130329.0,연
9,100,Y,사조동아원,8040.0,[기재정정]사업보고서 (2012.12),2.013051e+13,사조동아원,20130509.0,연


In [4]:
# rcp_no로 해당 사업보고서 url과 name 저장

url_code = []
url_name = []
length = len(data)

for t in range(length):
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + str(data['rcp_no'][t])[:-2]
    url_code.append(url)
    name = data['crp_nm'][t]
    url_name.append(name)

In [7]:
url_code[26:28]

['http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20130401002306',
 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20130401001124']

In [8]:
url_name.index('SK')

26

In [9]:
# Dart 의 IP차단을 막기 위해 갯수 설정
part_code = url_code[26:28]
part_name = url_name[26:28]

In [10]:
# 사업보고서 url에 그대로 들어가면 각 파트별 HTML코드가 없음.
# 따라서, 각 사업보고서의 HTML 코드에서 '이사의 경영진단 및 분석의견' url을 구한 뒤
# BeautifulSoup으로 HTML코드 partitioning

page_list = []
for code in part_code:
    time.sleep(1)
    page = BeautifulSoup(urlopen(code).read(), 'html.parser')  
    body = str(page.find('head')).split('이사의 경영진단 및 분석의견",')[1].split('cnt++')[0].split('viewDoc(')[1].split(')')[0].split(', ')
    body = [body[i][1:-1] for i in range(len(body))]   
    url_final = 'http://dart.fss.or.kr/report/viewer.do?rcpNo=' + body[0] + '&dcmNo=' + body[1] + '&eleId=' + body[2] + '&offset=' + body[3] + '&length=' + body[4] + '&dtd=' + body[5]
    page = BeautifulSoup(urlopen(url_final).read(), 'html.parser')
    page_list.append(page)
    print(url_final)
# page_list

http://dart.fss.or.kr/report/viewer.do?rcpNo=20130401002306&dcmNo=3711041&eleId=13&offset=1819576&length=469874&dtd=dart3.xsd
http://dart.fss.or.kr/report/viewer.do?rcpNo=20130401001124&dcmNo=3705683&eleId=13&offset=306434&length=46124&dtd=dart3.xsd


[<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
 
 <html style="border:0">
 <head>
 <title></title>
 <meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
 <meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
 <link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
 </head>
 <body bgcolor="#FFFFFF">
 <p><br/></p>
 <p class="section-1"><a name="toc1">V. 이사의 경영진단 및 분석의견</a></p>
 <p><br/></p>
 <p><br/></p>
 <table border="1" width="682">
 <colgroup>
 <col width="682"/>
 </colgroup>
 <tbody>
 <tr>
 <td height="296" valign="TOP" width="682">당사가 동 사업보고서에 기재하는 미래에 발생할 것으로 예측되는 경영활동 및 사건, 현상 등은 당해공시서류 작성시점의 재무성과 등에 대한 회사의 견해를 반영한 것입니다. 동 예측정보는 미래 경영환경과 관련된 다양한 가정에 기초하고 있으며, 동 예측정보는 결과적으로 부정확한 것으로 판명될 수 있습니다. <br/><br/>또한 이러한 가정들에는 예측정보에서 기재한 예상치와 실제 결과 간에 중요한 차이를 초래할 수 있는 위험, 불확실성 등을 포함하고 있습니다. 이러한 중요한 차이를 초래할 수 있는 요인에는 회사 내부경영과 관련된 요인과 외부환경에 관한 

In [22]:
import re
def remove_tag(content):             ## 태그를 제외하고 text만 가져오는 함수
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '\n', content)
    return cleantext
def remove_space(content):          # 개행문자인 \n과 \xa0제거
    cleans = re.compile('[\n]|[\xa0]')
    cleantext = re.sub(cleans, '', content)
    return cleantext

In [23]:
# partitioning한 코드에서 재무제표 등의 표를 제거하고 텍스트만 가져오기 위해 'table'태그를 제거하는 과정
# 표를 제거하기 위해 표를 크롤링해오는 함수를 만들어보고, 정규표현식을 써보고, HTML코드의 속성값으로 텍스트를 가져와보는 등 실패

# 함수로 만들기!!
page_partition = []
for page_char in page_list:
    whole_data = str(page_char).split('<')
    table_data = page_char.find_all('table')
    table_data_li = str(table_data).split('<')
    whole_data_copy = whole_data.copy()

    for table in table_data_li:
        if (table in whole_data) == True:
            whole_data_copy.remove(table)
        
    output = '<'.join(whole_data_copy)

    output1 = remove_tag(output)
    final = remove_space(output1)
    page_partition.append(final)

In [24]:
# page_partition

['V. 이사의 경영진단 및 분석의견1. 예측정보에 대한 주의사항본 자료는 미래에 대한 "예측정보"를 포함하고 있습니다.이는 과거가 아닌 미래의 사건과 관계된 것으로 회사의 향후 예상되는 경영현황 및 재무실적을 의미하고, 표현상으로는 \'예상\', \'전망\', \'계획\', \'기대\' 등과 같은 단어를 포함합니다."예측정보"는 그 성격상 불확실한 사건들을 언급하는데, 회사의 향후 경영현황 및 재무실적에 긍정적 또는 부정적으로 영향을 미칠 수 있는 불확실성에는 다음과 같은 것들이 포함됩니다.·환율, 이자율 등의 변동을 포함한 국내·외 금융 시장의 동향·사업의 처분, 매수 등을 포함한 회사의 전략적인 의사결정·반도체, LCD, 정보통신, 디지털미디어 등 회사가 영위하는 주요사업 분야의  예상치 못한 급격한 여건변화 ·기타 경영현황 및 재무실적에 영향을 미칠 수 있는 국내·외적 변화이러한 불확실성으로 인해 회사의 실제 미래실적은 "예측정보"에 명시적 또는 묵시적으로 포함된 내용과 중대한 차이가 있을 수 있음을 양지하시기 바랍니다.당사는 동 예측정보 작성시점 이후에 발생하는 위험 또는 불확실성을 반영하기 위하여 예측정보에 기재한 사항을 수정하는 정정보고서를 공시할 의무는 없습니다.2. 개요2010년 당사는 선진국 경기회복 지연, 유럽 재정위기, 환율변동성 확대 등 어려운 여건에서도 사상 최대 매출 및 영업이익을 달성하였습니다. 특히, 메모리, TV, HHP 등주력 사업에서 시장지배력을 확대하는 한편, 제품 차별화를 통하여 시장경쟁의 심화에도 불구하고 견고한 이익을 달성하였습니다. TV 부문은 시장경쟁 심화에 따른 판가하락에도 불구하고, 3D/Smart TV 출시를 통해 프리미엄 시장을 장악하였고, 경쟁사에 비해 앞선 제품력과 디자인으로 2006년이후 지속 1위를 유지하였습니다. 휴대폰 부문은 Galaxy S가 판매량 10백만대를 돌파하며 스마트폰의 폭발적 성장을 견인하였고, 서남아ㆍ중남미등 신흥시장에서의 고성장으로 세계시장 점유율 20%를 돌파하여, 사업위상을 지

In [ ]:
# txt file로 저장

result_texts = zip(part_name, page_partition)
for name,text in result_texts:
    file = open('C:/Users/PC/Desktop/2010_text/%s 사업보고서.txt' %name, 'w', encoding='utf-8')
    file.write(text)
    file.close()